In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [7]:
split = 7869

In [8]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [9]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:split],x_train_text[:split],x_train_mocap[:split]], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6290:],x_train_text[6290:],x_train_mocap[6290:]], batch_size=32)
# np.save('speech_text_mocap-batch7869_npy', 
#          np.array(predict).reshape(3, 3749).T)


In [10]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[6290:],x_train_text[6290:],x_train_mocap[6290:]], batch_size=32)
    np.save('npy7869/stm_npy-glove-7869batch'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3749).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/stm_glove_multi7869-batch-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 94s 15ms/step - loss: 1.6092 - v_loss: 0.6122 - a_loss: 0.4385 - d_loss: 0.5574 - v_ccc: 0.3872 - a_ccc: 0.5612 - d_ccc: 0.4424 - val_loss: 2.0391 - val_v_loss: 0.6831 - val_a_loss: 0.7038 - val_d_loss: 0.6712 - val_v_ccc: 0.3222 - val_a_ccc: 0.3030 - val_d_ccc: 0.3357
Epoch 2/50
6295/6295 [==============================] - 85s 13ms/step - loss: 1.0704 - v_loss: 0.3414 - a_loss: 0.3154 - d_loss: 0.4132 - v_ccc: 0.6584 - a_ccc: 0.6845 - d_ccc: 0.5867 - val_loss: 1.7113 - val_v_loss: 0.6074 - val_a_loss: 0.5196 - val_d_loss: 0.5913 - val_v_ccc: 0.3925 - val_a_ccc: 0.4844 - val_d_ccc: 0.4118
Epoch 3/50
6295/6295 [==============================] - 84s 13ms/step - loss: 0.9326 - v_loss: 0.2654 - a_loss: 0.2957 - d_loss: 0.3714 - v_ccc: 0.7346 - a_ccc: 0.7043 - d_ccc: 0.6285 - val_loss: 1.5394 - val_v_loss: 0

Epoch 7/50
6295/6295 [==============================] - 88s 14ms/step - loss: 0.6265 - v_loss: 0.1569 - a_loss: 0.2247 - d_loss: 0.2448 - v_ccc: 0.8430 - a_ccc: 0.7752 - d_ccc: 0.7553 - val_loss: 1.5020 - val_v_loss: 0.5196 - val_a_loss: 0.4435 - val_d_loss: 0.5577 - val_v_ccc: 0.4845 - val_a_ccc: 0.5634 - val_d_ccc: 0.4501
Epoch 8/50
6295/6295 [==============================] - 88s 14ms/step - loss: 0.5737 - v_loss: 0.1406 - a_loss: 0.2070 - d_loss: 0.2260 - v_ccc: 0.8593 - a_ccc: 0.7931 - d_ccc: 0.7739 - val_loss: 1.5137 - val_v_loss: 0.5322 - val_a_loss: 0.4357 - val_d_loss: 0.5605 - val_v_ccc: 0.4697 - val_a_ccc: 0.5707 - val_d_ccc: 0.4459
Epoch 9/50
6295/6295 [==============================] - 87s 14ms/step - loss: 0.5297 - v_loss: 0.1306 - a_loss: 0.1928 - d_loss: 0.2066 - v_ccc: 0.8696 - a_ccc: 0.8071 - d_ccc: 0.7936 - val_loss: 1.5761 - val_v_loss: 0.5415 - val_a_loss: 0.4749 - val_d_loss: 0.5840 - val_v_ccc: 0.4656 - val_a_ccc: 0.5355 - val_d_ccc: 0.4228
Epoch 10/50
6295/6295 

2170/2170 [==============================] - 5s 2ms/step
[0.46855059266090393, 0.5975888967514038, 0.48849934339523315]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 93s 15ms/step - loss: 1.5900 - v_loss: 0.6413 - a_loss: 0.4203 - d_loss: 0.5276 - v_ccc: 0.3584 - a_ccc: 0.5794 - d_ccc: 0.4722 - val_loss: 2.0312 - val_v_loss: 0.6657 - val_a_loss: 0.7207 - val_d_loss: 0.6514 - val_v_ccc: 0.3412 - val_a_ccc: 0.2808 - val_d_ccc: 0.3468
Epoch 2/50
6295/6295 [==============================] - 87s 14ms/step - loss: 1.1034 - v_loss: 0.3652 - a_loss: 0.3194 - d_loss: 0.4190 - v_ccc: 0.6346 - a_ccc: 0.6809 - d_ccc: 0.5811 - val_loss: 1.5975 - val_v_loss: 0.6421 - val_a_loss: 0.4708 - val_d_loss: 0.4947 - val_v_ccc: 0.3622 - val_a_ccc: 0.5349 - val_d_ccc: 0.5054
Epoch 3/50
6295/6295 [==============================] - 86s 14ms/step - loss: 0.9568 - v_loss: 0.2745 - a_loss: 0.3040 - d_loss: 0.3780 - v_ccc: 0.7254 - a_ccc: 0.6959 - d_ccc: 0

6295/6295 [==============================] - 86s 14ms/step - loss: 0.5374 - v_loss: 0.1326 - a_loss: 0.1955 - d_loss: 0.2093 - v_ccc: 0.8673 - a_ccc: 0.8045 - d_ccc: 0.7908 - val_loss: 1.5242 - val_v_loss: 0.5282 - val_a_loss: 0.4600 - val_d_loss: 0.5516 - val_v_ccc: 0.4728 - val_a_ccc: 0.5492 - val_d_ccc: 0.4538
Epoch 10/50
6295/6295 [==============================] - 85s 14ms/step - loss: 0.4937 - v_loss: 0.1229 - a_loss: 0.1812 - d_loss: 0.1897 - v_ccc: 0.8772 - a_ccc: 0.8187 - d_ccc: 0.8104 - val_loss: 1.5322 - val_v_loss: 0.5243 - val_a_loss: 0.4771 - val_d_loss: 0.5531 - val_v_ccc: 0.4816 - val_a_ccc: 0.5335 - val_d_ccc: 0.4526
Epoch 11/50
6295/6295 [==============================] - 85s 14ms/step - loss: 0.4527 - v_loss: 0.1132 - a_loss: 0.1665 - d_loss: 0.1735 - v_ccc: 0.8871 - a_ccc: 0.8335 - d_ccc: 0.8267 - val_loss: 1.5314 - val_v_loss: 0.5668 - val_a_loss: 0.4683 - val_d_loss: 0.5266 - val_v_ccc: 0.4425 - val_a_ccc: 0.5462 - val_d_ccc: 0.4799
Epoch 12/50
6295/6295 [========

6295/6295 [==============================] - 94s 15ms/step - loss: 1.5765 - v_loss: 0.6216 - a_loss: 0.4204 - d_loss: 0.5341 - v_ccc: 0.3781 - a_ccc: 0.5795 - d_ccc: 0.4659 - val_loss: 2.1019 - val_v_loss: 0.7025 - val_a_loss: 0.7401 - val_d_loss: 0.6631 - val_v_ccc: 0.3009 - val_a_ccc: 0.2623 - val_d_ccc: 0.3349
Epoch 2/50
6295/6295 [==============================] - 86s 14ms/step - loss: 1.1093 - v_loss: 0.3630 - a_loss: 0.3249 - d_loss: 0.4216 - v_ccc: 0.6372 - a_ccc: 0.6753 - d_ccc: 0.5783 - val_loss: 1.5845 - val_v_loss: 0.5612 - val_a_loss: 0.4752 - val_d_loss: 0.5551 - val_v_ccc: 0.4407 - val_a_ccc: 0.5292 - val_d_ccc: 0.4456
Epoch 3/50
6295/6295 [==============================] - 85s 14ms/step - loss: 0.9524 - v_loss: 0.2720 - a_loss: 0.3032 - d_loss: 0.3770 - v_ccc: 0.7279 - a_ccc: 0.6969 - d_ccc: 0.6228 - val_loss: 1.5008 - val_v_loss: 0.5632 - val_a_loss: 0.4474 - val_d_loss: 0.5000 - val_v_ccc: 0.4384 - val_a_ccc: 0.5567 - val_d_ccc: 0.5041
Epoch 4/50
6295/6295 [===========

...

6295/6295 [==============================] - 93s 15ms/step - loss: 0.7779 - v_loss: 0.2010 - a_loss: 0.2630 - d_loss: 0.3136 - v_ccc: 0.7988 - a_ccc: 0.7369 - d_ccc: 0.6864 - val_loss: 1.4733 - val_v_loss: 0.5315 - val_a_loss: 0.4379 - val_d_loss: 0.5241 - val_v_ccc: 0.4780 - val_a_ccc: 0.5683 - val_d_ccc: 0.4805
Epoch 6/50
6295/6295 [==============================] - 93s 15ms/step - loss: 0.7164 - v_loss: 0.1809 - a_loss: 0.2483 - d_loss: 0.2872 - v_ccc: 0.8191 - a_ccc: 0.7517 - d_ccc: 0.7128 - val_loss: 1.4920 - val_v_loss: 0.5206 - val_a_loss: 0.4230 - val_d_loss: 0.5587 - val_v_ccc: 0.4803 - val_a_ccc: 0.5807 - val_d_ccc: 0.4469
Epoch 7/50
6295/6295 [==============================] - 93s 15ms/step - loss: 0.6552 - v_loss: 0.1630 - a_loss: 0.2290 - d_loss: 0.2630 - v_ccc: 0.8369 - a_ccc: 0.7710 - d_ccc: 0.7369 - val_loss: 1.4717 - val_v_loss: 0.5191 - val_a_loss: 0.4316 - val_d_loss: 0.5285 - val_v_ccc: 0.4850 - val_a_ccc: 0.5689 - val_d_ccc: 0.4744
Epoch 8/50
6295/6295 [===========

Epoch 13/50
6295/6295 [==============================] - 95s 15ms/step - loss: 0.3965 - v_loss: 0.1015 - a_loss: 0.1456 - d_loss: 0.1496 - v_ccc: 0.8986 - a_ccc: 0.8545 - d_ccc: 0.8504 - val_loss: 1.5542 - val_v_loss: 0.5420 - val_a_loss: 0.4827 - val_d_loss: 0.5421 - val_v_ccc: 0.4595 - val_a_ccc: 0.5251 - val_d_ccc: 0.4611
Epoch 14/50
6295/6295 [==============================] - 95s 15ms/step - loss: 0.3710 - v_loss: 0.0942 - a_loss: 0.1383 - d_loss: 0.1383 - v_ccc: 0.9058 - a_ccc: 0.8615 - d_ccc: 0.8617 - val_loss: 1.5608 - val_v_loss: 0.5366 - val_a_loss: 0.4842 - val_d_loss: 0.5668 - val_v_ccc: 0.4740 - val_a_ccc: 0.5264 - val_d_ccc: 0.4388
Epoch 15/50
6295/6295 [==============================] - 95s 15ms/step - loss: 0.3488 - v_loss: 0.0897 - a_loss: 0.1327 - d_loss: 0.1267 - v_ccc: 0.9105 - a_ccc: 0.8673 - d_ccc: 0.8733 - val_loss: 1.5512 - val_v_loss: 0.5336 - val_a_loss: 0.4722 - val_d_loss: 0.5670 - val_v_ccc: 0.4761 - val_a_ccc: 0.5343 - val_d_ccc: 0.4385
Epoch 16/50
6295/62

6295/6295 [==============================] - 96s 15ms/step - loss: 0.7213 - v_loss: 0.1809 - a_loss: 0.2480 - d_loss: 0.2924 - v_ccc: 0.8190 - a_ccc: 0.7522 - d_ccc: 0.7075 - val_loss: 1.4921 - val_v_loss: 0.5366 - val_a_loss: 0.4280 - val_d_loss: 0.5442 - val_v_ccc: 0.4689 - val_a_ccc: 0.5798 - val_d_ccc: 0.4593
Epoch 7/50
6295/6295 [==============================] - 95s 15ms/step - loss: 0.6566 - v_loss: 0.1653 - a_loss: 0.2341 - d_loss: 0.2572 - v_ccc: 0.8348 - a_ccc: 0.7658 - d_ccc: 0.7427 - val_loss: 1.4779 - val_v_loss: 0.5229 - val_a_loss: 0.4325 - val_d_loss: 0.5438 - val_v_ccc: 0.4864 - val_a_ccc: 0.5770 - val_d_ccc: 0.4587
Epoch 8/50
6295/6295 [==============================] - 96s 15ms/step - loss: 0.5978 - v_loss: 0.1497 - a_loss: 0.2151 - d_loss: 0.2328 - v_ccc: 0.8503 - a_ccc: 0.7848 - d_ccc: 0.7671 - val_loss: 1.4955 - val_v_loss: 0.5242 - val_a_loss: 0.4548 - val_d_loss: 0.5325 - val_v_ccc: 0.4813 - val_a_ccc: 0.5517 - val_d_ccc: 0.4715
Epoch 9/50
6295/6295 [===========

Epoch 14/50
6295/6295 [==============================] - 94s 15ms/step - loss: 0.3645 - v_loss: 0.0946 - a_loss: 0.1336 - d_loss: 0.1362 - v_ccc: 0.9053 - a_ccc: 0.8664 - d_ccc: 0.8638 - val_loss: 1.5300 - val_v_loss: 0.5268 - val_a_loss: 0.4643 - val_d_loss: 0.5431 - val_v_ccc: 0.4729 - val_a_ccc: 0.5380 - val_d_ccc: 0.4591
Epoch 15/50
2170/2170 [==============================] - 6s 3ms/step
[0.47740793228149414, 0.5833892822265625, 0.49285319447517395]
17
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 107s 17ms/step - loss: 1.5814 - v_loss: 0.6365 - a_loss: 0.4198 - d_loss: 0.5244 - v_ccc: 0.3633 - a_ccc: 0.5799 - d_ccc: 0.4754 - val_loss: 2.0798 - val_v_loss: 0.6864 - val_a_loss: 0.7385 - val_d_loss: 0.6532 - val_v_ccc: 0.3133 - val_a_ccc: 0.2651 - val_d_ccc: 0.3418
Epoch 2/50
6295/6295 [==============================] - 95s 15ms/step - loss: 1.1099 - v_loss: 0.3789 - a_loss: 0.3169 - d_loss: 0.4137 - v_ccc: 0.6210 - a_ccc: 0.

6295/6295 [==============================] - 94s 15ms/step - loss: 0.5546 - v_loss: 0.1360 - a_loss: 0.2000 - d_loss: 0.2190 - v_ccc: 0.8640 - a_ccc: 0.8003 - d_ccc: 0.7811 - val_loss: 1.6552 - val_v_loss: 0.5521 - val_a_loss: 0.5248 - val_d_loss: 0.5973 - val_v_ccc: 0.4572 - val_a_ccc: 0.4826 - val_d_ccc: 0.4049
Epoch 10/50
6295/6295 [==============================] - 96s 15ms/step - loss: 0.4998 - v_loss: 0.1264 - a_loss: 0.1781 - d_loss: 0.1955 - v_ccc: 0.8736 - a_ccc: 0.8219 - d_ccc: 0.8047 - val_loss: 1.4988 - val_v_loss: 0.5111 - val_a_loss: 0.4574 - val_d_loss: 0.5498 - val_v_ccc: 0.5005 - val_a_ccc: 0.5488 - val_d_ccc: 0.4519
Epoch 11/50
6295/6295 [==============================] - 96s 15ms/step - loss: 0.4593 - v_loss: 0.1147 - a_loss: 0.1694 - d_loss: 0.1752 - v_ccc: 0.8853 - a_ccc: 0.8306 - d_ccc: 0.8248 - val_loss: 1.5017 - val_v_loss: 0.5201 - val_a_loss: 0.4505 - val_d_loss: 0.5501 - val_v_ccc: 0.4922 - val_a_ccc: 0.5527 - val_d_ccc: 0.4534
Epoch 12/50
6295/6295 [========